In [ ]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pandas as pd
import numpy as np
import datetime
import os
import itertools

In [ ]:
def findTime(x):
    return x.time()
def findDate(x):
    return x.date()


filelist = os.listdir('measurement')
filelist.sort()
startpoint = np.where(np.array(filelist) == '16157')[0][0]
filelist = filelist[startpoint:]

df = pd.DataFrame()
for fileid in filelist:
    data = pd.read_csv('measurement/'+fileid+'/tab_ldt.csv')
    df = pd.concat([df,data])


for roadid in df['DEVICEID'].unique(): # 对测试路段循环
    road_data = df[df['DEVICEID'] == roadid].copy()
    road_data['FROMTIME'] = pd.to_datetime(road_data['FROMTIME'])
    road_data.index = road_data['FROMTIME']
    road_data['FROMTIME-Time'] = road_data['FROMTIME'].apply(findTime)  # 找到时间
    road_data['FROMTIME-Date'] = road_data['FROMTIME'].apply(findDate)  # 找到日期
    road_data.to_csv('train_data/data/'+'%s.csv'%roadid)
    # 对每个日期进行绘图
    plt.figure(figsize= (20,5))
    plt.title(str(roadid))
    for date in road_data['FROMTIME-Date'].unique():
        df_temp = road_data[road_data['FROMTIME-Date'] == date]
        data3 = df_temp.resample(datetime.timedelta(seconds=5 * 60)).sum()
        data3.index = pd.Series(data3.index).apply(findTime)
        plt.plot(data3.index,data3['FLOW'],label=date)
    plt.legend()

In [ ]:
road_data = pd.read_csv('train_data/data/5696.csv')
road_data['FROMTIME'] = pd.to_datetime(road_data['FROMTIME'])
road_data.index = road_data['FROMTIME']
road_data['FROMTIME-Time'] = road_data['FROMTIME'].apply(findTime)  # 找到时间
road_data['FROMTIME-Date'] = road_data['FROMTIME'].apply(findDate)  # 找到日期
road_data
# 对每个日期进行绘图
plt.figure(figsize= (20,5))
plt.title(str(5696))
for date in road_data['FROMTIME-Date'].unique():
    df_temp = road_data[road_data['FROMTIME-Date'] == date]
    data3 = df_temp.resample(datetime.timedelta(seconds=5 * 60)).sum()
    data3.index = pd.Series(data3.index).apply(findTime)
    plt.plot(data3.index,data3['FLOW'],label=date)
plt.legend()

### LSTM预测

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.models import load_model
from keras.utils.vis_utils import plot_model
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

In [ ]:
# 读取文件
xianquan = road_data.resample(datetime.timedelta(seconds=5 * 60)).sum()
df = xianquan.copy()
# 打印头部
df.head()

In [ ]:
#筛选出题目要求的7点到22点的数据
df['FROMTIME'] = pd.to_datetime(df.index)
df['FROMTIME-Time'] = df["FROMTIME"].apply(findTime)  # 找到时间
df['FROMTIME-Date'] = df["FROMTIME"].apply(findDate)  # 找到日期
df=df[(pd.to_datetime(df["FROMTIME-Time"],format = '%H:%M:%S')>= pd.to_datetime('07:00:00',format = '%H:%M:%S'))&(pd.to_datetime(df["FROMTIME-Time"],format = '%H:%M:%S')<= pd.to_datetime('22:00:00',format = '%H:%M:%S'))]

In [ ]:
#训练集：2019年7月周六周日7月6日，7日；13日，14日；20日，21日；27日，28日；其余天数为工作日
#待预测的天数：8月3日，8月4日为周六周日，其余为工作日
weekend_list=["2019-07-06","2019-07-07","2019-07-13","2019-07-14","2019-07-20","2019-07-21","2019-07-27","2019-07-28"]

df_weekend=pd.DataFrame()
for day in weekend_list:
    temp=df[pd.to_datetime(df['FROMTIME-Date'],format = '%Y-%m-%d')==pd.to_datetime(day,format = '%Y-%m-%d')]
    df_weekend=pd.concat([df_weekend,temp])
    #df_weekend包括所有的周末数据
df_weekend.index = df_weekend['FROMTIME']
df_weekend.to_csv('train_data/lstm/weekend_5696')

In [ ]:
weekday_list=["2019-07-01","2019-07-02","2019-07-03","2019-07-04","2019-07-05","2019-07-08","2019-07-09","2019-07-10",
             "2019-07-11","2019-07-12","2019-07-15","2019-07-16","2019-07-17","2019-07-18","2019-07-19","2019-07-22",
             "2019-07-23","2019-07-24","2019-07-25","2019-07-26","2019-07-29","2019-07-30","2019-07-31"]
df_weekday=pd.DataFrame()
for day in weekday_list:
    temp=df[pd.to_datetime(df['FROMTIME-Date'],format = '%Y-%m-%d')==pd.to_datetime(day,format = '%Y-%m-%d')]
    df_weekday=pd.concat([df_weekday,temp])
    #df_weekday包括所有的工作日数据

df_weekday.index = df_weekday['FROMTIME']
df_weekday.to_csv('train_data/lstm/weekday_5696')

In [ ]:
plt.figure(figsize= (20,5))
plt.title(str('5696'))
plt.plot(df_weekend.index,df_weekend['COUNT'])
plt.plot(df_weekday.index,df_weekday['COUNT'])

#### 数据预处理函数

In [ ]:
#对数据进行处理
def process_data(train, test, lags):
    """Process data
    Reshape and split train\test data.

    # Arguments
        train: String, name of .csv train file.
        test: String, name of .csv test file.
        lags: integer, time lag.
    # Returns
        X_train: ndarray.
        y_train: ndarray.
        X_test: ndarray.
        y_test: ndarray.
        scaler: StandardScaler.
    """
    attr = 'COUNT'
    df1 = pd.read_csv(train, encoding='utf-8').fillna(0)
    df2 = pd.read_csv(test, encoding='utf-8').fillna(0)

    # scaler = StandardScaler().fit(df1[attr].values)

    scaler = MinMaxScaler(feature_range=(0, 1)).fit(df1[attr].values.reshape(-1, 1))
    flow1 = scaler.transform(df1[attr].values.reshape(-1, 1))
    flow2 = scaler.transform(df2[attr].values.reshape(-1, 1))

    train, test = [], []
    #关于train和test的划分要清楚
    #这里是将最后一周的数据作为测试集
#     for i in range(lags, len(flow1)):
#         train.append(flow1[i - lags: i + 1])
#     for i in range(lags, len(flow2)):
#         test.append(flow2[i - lags: i + 1])

    #训练集划分的时候注意是每天7点开始，22点结束，不能用22点预测7点的，一天一共16个小时
    for i in range(lags, len(flow1),12):
        if (i/12)%16==0:
            pass
        else:
            #0:i+1实际输出的长度是i,比如i=12时,输出的是flow[0]...flow[12]。所以是输出了13个数据
            temp=flow1[i - lags: i + 12]
            if len(temp)==24:
                train.append(temp)
                
    for i in range(lags, len(flow2),12):
        if (i/12)%16==0:
            pass
        else:
            temp=flow1[i - lags: i + 12]
            if len(temp)==24:
                test.append(temp)

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)
    #train[:,-1] train这个二维的数据，逗号分隔开的前面的":"是说取全部的行，逗号后面的-1是说取最后一列。
    #train[:, :-1]是指除去数据的最后一列剩下的数据
    X_train = train[:,0:12]
    y_train = train[:,-12:]
    X_test = test[:,0:12]
    y_test = test[:,-12:]
    
    return X_train, y_train, X_test, y_test, scaler


#对数据进行处理
def process_data2(train, test, lags):
    attr = 'COUNT'
    df1 = pd.read_csv(train, encoding='utf-8').fillna(0)
    df2 = pd.read_csv(test, encoding='utf-8').fillna(0)

    # scaler = StandardScaler().fit(df1[attr].values)

    scaler = MinMaxScaler(feature_range=(0, 1)).fit(df1[attr].values.reshape(-1, 1))
    flow1 = scaler.transform(df1[attr].values.reshape(-1, 1))
    flow2 = scaler.transform(df2[attr].values.reshape(-1, 1))

    train, test = [], []
    #关于train和test的划分要清楚
    #这里是将最后一周的数据作为测试集
    for i in range(lags, len(flow1)):
        train.append(flow1[i - lags: i + 1])
    for i in range(lags, len(flow2)):
        test.append(flow2[i - lags: i + 1])

    train = np.array(train)
    test = np.array(test)
    np.random.shuffle(train)
    #train[:,-1] train这个二维的数据，逗号分隔开的前面的":"是说取全部的行，逗号后面的-1是说取最后一列。
    #train[:, :-1]是指除去数据的最后一列剩下的数据
    X_train = train[:, :-1]
    y_train = train[:,-1] 
    X_test = test[:, :-1]
    y_test = test[:,-1]

    return  X_train, y_train, X_test, y_test, scaler,train,test

In [ ]:
#定义LSTM模型
def get_lstm(units):
    """LSTM(Long Short-Term Memory)
    Build LSTM Model.

    # Arguments
        units: List(int), number of input, output and hidden units.
    # Returns
        model: Model, nn model.
    """

    model = Sequential()
    model.add(LSTM(units[1], input_shape=(units[0], 1), return_sequences=True))
    model.add(LSTM(units[2]))
    model.add(Dropout(0.2))
    model.add(Dense(units=units[3],activation='linear'))

    return model

In [ ]:
"""
训练，LSTM、GRU按照正常的RNN网络进行训练。使用train_model()函数训练。

使用RMSprop(lr=0.001, rho=0.9, epsilon=1e-06)作为优化器，batch_szie为256，lags为12(即时滞长度为一个小时)。
"""
def train_model(model, X_train, y_train, name, config):
    """train
    train a single model.

    # Arguments
        model: Model, NN model to train.
        X_train: ndarray(number, lags), Input data for train.
        y_train: ndarray(number, ), result data for train.
        name: String, name of model.
        config: Dict, parameter for train.
    """

    model.compile(loss="mse", optimizer="rmsprop", metrics=['mape'])
    # early = EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
    hist = model.fit(
        X_train, y_train,
        batch_size=config["batch"],
        epochs=config["epochs"],
        validation_split=0.15)#原先的参数是0.05

    model.save('train_data/model/' + name + '.h5')
    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv('train_data/model/' + name + ' loss.csv', encoding='utf-8', index=False)

In [ ]:
def MAPE(y_true, y_pred):
    """Mean Absolute Percentage Error
    Calculate the mape.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    # Returns
        mape: Double, result data for train.
    """

    y = [x for x in y_true if x > 0]
    y_pred = [y_pred[i] for i in range(len(y_true)) if y_true[i] > 0]

    num = len(y_pred)
    sums = 0

    for i in range(num):
        tmp = abs(y[i] - y_pred[i]) / y[i]
        sums += tmp
    if num!=0:
        mape = sums * (100 / num)
    else:
        mape=0
    return mape


def eva_regress(y_true, y_pred):
    """Evaluation
    evaluate the predicted resul.

    # Arguments
        y_true: List/ndarray, ture data.
        y_pred: List/ndarray, predicted data.
    """

    mape = MAPE(y_true, y_pred)
    vs = metrics.explained_variance_score(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print('explained_variance_score:%f' % vs)
    print('mape:%f%%' % mape)
    print('mae:%f' % mae)
    print('mse:%f' % mse)
    print('rmse:%f' % math.sqrt(mse))
    print('r2:%f' % r2)

In [ ]:
#周末数据的训练
df_weekend=pd.read_csv('train_data/lstm/weekend_5696')
df_weekday=pd.read_csv('train_data/lstm/weekday_5696')
data1=df_weekend[(pd.to_datetime(df_weekend.index,format = '%Y-%m-%d %H:%M:%S')<= pd.to_datetime('2019-07-26 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data2=df_weekend[(pd.to_datetime(df_weekend.index,format = '%Y-%m-%d %H:%M:%S')> pd.to_datetime('2019-07-26 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data1.to_csv('train_data/train.csv')
data2.to_csv('train_data/test.csv')

In [ ]:
#数据处理
lag = 12
file1 = 'train_data/train.csv'
file2 = 'train_data/test.csv'

X_train, y_train, X_test, y_test, scaler= process_data(file1, file2, lag)

In [ ]:
y_train=y_train.reshape(y_train.shape[0],y_train.shape[1])
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])

In [ ]:
#训练模型
units=[X_train.shape[1],50,50,y_train.shape[1]]
model=get_lstm(units)
name='lstm'
config={"batch":100,"epochs":10000}
train_model(model, X_train, y_train, name, config)

In [ ]:
lstm = load_model('train_data/model/lstm.h5')

model=lstm
name='LSTM'

lag = 12
file1 = 'train_data/train.csv'
file2 = 'train_data/test.csv'
_, _, X_test, y_test, scaler = process_data(file1, file2, lag)
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])
y_test = scaler.inverse_transform(y_test)

y_preds = []

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
file = 'train_data/model' + name + '.png'
plot_model(model, to_file=file, show_shapes=True)
predicted = model.predict(X_test)
predicted = scaler.inverse_transform(predicted)
# y_preds.append(predicted)#[:12]
y_preds=predicted
print(name)


In [ ]:
for i in range(len(y_test)):
    plt.figure()
    plt.plot(y_test[i])
    plt.plot(y_preds[i])
    eva_regress(y_test[i], y_preds[i])

#### 工作日

In [ ]:
#工作日的数据进行训练，前15天训练集，后5天测试集
df_weekday=pd.read_csv('train_data/lstm/weekday_5696')
df_weekday.index=df_weekday["FROMTIME"]
data5=df_weekday[(pd.to_datetime(df_weekday.index,format = '%Y-%m-%d %H:%M:%S')<= pd.to_datetime('2019-07-24 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data6=df_weekday[(pd.to_datetime(df_weekday.index,format = '%Y-%m-%d %H:%M:%S')> pd.to_datetime('2019-07-24 22:00:00',format = '%Y-%m-%d %H:%M:%S'))]
data5.to_csv('train_data/train_5696_wd.csv')
data6.to_csv('train_data/test_5696_wd.csv')

In [ ]:
#数据处理
lag = 12
file1 = 'train_data/train_5696_wd.csv'
file2 = 'train_data/test_5696_wd.csv'

X_train, y_train, X_test, y_test, scaler= process_data(file1, file2, lag)

In [ ]:
y_train=y_train.reshape(y_train.shape[0],y_train.shape[1])
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])

In [ ]:
#训练模型
units=[X_train.shape[1],50,50,y_train.shape[1]]
model=get_lstm(units)
name='lstm'
config={"batch":100,"epochs":10000}
train_model(model, X_train, y_train, name, config)

In [ ]:
lstm = load_model('train_data/model/lstm.h5')

model=lstm
name='LSTM'

lag = 12
file1 = 'train_data/train_5696_wd.csv'
file2 = 'train_data/test_5696_wd.csv'
_, _, X_test, y_test, scaler = process_data(file1, file2, lag)
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1])
y_test = scaler.inverse_transform(y_test)

y_preds = []

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
file = 'train_data/model' + name + '.png'
plot_model(model, to_file=file, show_shapes=True)
predicted = model.predict(X_test)
predicted = scaler.inverse_transform(predicted)
# y_preds.append(predicted)#[:12]
y_preds=predicted
print(name)

In [ ]:
for i in range(len(y_test)):
    plt.figure()
    plt.plot(y_test[i])
    plt.plot(y_preds[i])
    eva_regress(y_test[i], y_preds[i])